# Real-Time Network Intrusion Detection System (NIDS)

In this notebook, we prepare flow-based network traffic data for training the anomaly detection component of our Real-Time NIDS. We begin by loading and exploring the **CSE-CIC-IDS2018** dataset, then extract and engineer key flow-level features such as `packet_rate` and `byte_rate`. These features are used to train an unsupervised machine learning model **(IsolationForest)** that can detect **anomalies** in real-time traffic. The resulting model will be integrated into the RealTimeNIDS system to identify suspicious activity based on flow behavior.

**Imports**:

In [25]:
import os

import pandas as pd
import numpy as np
import joblib

from sklearn.ensemble import IsolationForest

## Exploratory data analysis (EDA)

In this section, we examine the structure and quality of the dataset to understand the available features, identify missing or inconsistent values, and ensure the data is suitable for feature engineering. This step helps us prepare the dataset for training a robust anomaly detection model.

**Load Dataset**:

In [2]:
df = pd.read_csv('../dataset/02-14-2018.csv')
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,14/02/2018 08:31:01,112641719,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [3]:
df.columns

Index(['Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Tot Fwd Pkts',
       'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags',
       'Bwd URG Flags', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Min', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'FIN Flag Cnt', 'SYN Flag Cnt',
       'RST Flag Cnt', 'PSH Flag Cnt', 'ACK Flag Cnt', 'URG Flag Cnt',
       'CWE Flag Count', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
      

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 80 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   Dst Port           1048575 non-null  int64  
 1   Protocol           1048575 non-null  int64  
 2   Timestamp          1048575 non-null  object 
 3   Flow Duration      1048575 non-null  int64  
 4   Tot Fwd Pkts       1048575 non-null  int64  
 5   Tot Bwd Pkts       1048575 non-null  int64  
 6   TotLen Fwd Pkts    1048575 non-null  int64  
 7   TotLen Bwd Pkts    1048575 non-null  int64  
 8   Fwd Pkt Len Max    1048575 non-null  int64  
 9   Fwd Pkt Len Min    1048575 non-null  int64  
 10  Fwd Pkt Len Mean   1048575 non-null  float64
 11  Fwd Pkt Len Std    1048575 non-null  float64
 12  Bwd Pkt Len Max    1048575 non-null  int64  
 13  Bwd Pkt Len Min    1048575 non-null  int64  
 14  Bwd Pkt Len Mean   1048575 non-null  float64
 15  Bwd Pkt Len Std    1048575 non-n

In [5]:
df.describe()

C:\Users\emrev\PycharmProjects\RealTimeNIDS\.venv\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\emrev\PycharmProjects\RealTimeNIDS\.venv\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,...,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06,1.048575e+06
mean,4.876262e+03,8.107557e+00,6.255555e+06,6.206622e+00,7.211191e+00,4.479936e+02,4.521803e+03,1.745736e+02,8.389535e+00,3.879579e+01,...,2.793536e+00,2.327970e+01,5.152449e+04,2.136151e+04,8.789157e+04,3.995477e+04,3.101206e+06,7.297218e+05,4.812391e+06,2.126920e+06
std,1.444344e+04,4.460625e+00,1.260291e+09,4.447851e+01,1.048682e+02,1.573541e+04,1.515021e+05,2.876713e+02,1.948279e+01,5.331882e+01,...,5.557106e+00,1.106185e+01,5.815586e+05,2.186405e+05,7.395725e+05,5.602693e+05,5.414780e+08,3.820031e+08,1.522117e+09,1.817013e+07
min,0.000000e+00,0.000000e+00,-9.190110e+11,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.200000e+01,6.000000e+00,7.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.300000e+01,6.000000e+00,1.023000e+03,2.000000e+00,1.000000e+00,3.600000e+01,5.500000e+01,3.400000e+01,0.000000e+00,2.566667e+01,...,0.000000e+00,2.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,4.430000e+02,6.000000e+00,4.066690e+05,7.000000e+00,6.000000e+00,4.550000e+02,7.680000e+02,1.990000e+02,0.000000e+00,5.550000e+01,...,4.000000e+00,3.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,6.553300e+04,1.700000e+01,1.200000e+08,5.115000e+03,9.198000e+03,8.591554e+06,1.339773e+07,6.444000e+04,1.460000e+03,1.121703e+04,...,1.031000e+03,4.800000e+01,1.102401e+08,5.723446e+07,1.102401e+08,1.102401e+08,3.394503e+11,2.432682e+11,9.797810e+11,1.260300e+10


**Check Missing Values and Duplicates**:

In [6]:
df.isna().sum().sort_values(ascending=False)

Flow Byts/s     2277
Dst Port           0
Timestamp          0
Protocol           0
Tot Fwd Pkts       0
                ... 
Idle Mean          0
Idle Std           0
Idle Max           0
Idle Min           0
Label              0
Length: 80, dtype: int64

In [7]:
len(df)

1048575

In [8]:
df.duplicated().sum()

np.int64(225628)

**Seperate to `benign` and `attack`**:

In [9]:
df_benign = df[df['Label'].str.strip().str.lower() == 'benign']
df_benign['Label'].nunique()

1

In [10]:
df_attack = df[df['Label'].str.strip().str.lower() != 'benign']
df_attack['Label'].nunique()

2

## Preprocessing

In this section, we prepare the dataset for model training by filtering out non-benign traffic, engineering flow-level features, and removing unnecessary columns. This ensures that the training data represents only normal network behavior, allowing the anomaly detection model to accurately identify deviations during real-time analysis.

In [17]:
def extract_features(df):
    # Make a full copy to avoid SettingWithCopyWarning
    df = df.copy()

    # Convert to numeric safely
    numeric_cols = ['Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'TotLen Bwd Pkts']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Drop rows with NaNs in key columns
    df.dropna(subset=numeric_cols, inplace=True)

    # Duration in seconds
    df['flow_duration'] = df['Flow Duration'] / 1e6

    # Total Packet Count and Size
    df['packet_count'] = df['Tot Fwd Pkts'] + df['Tot Bwd Pkts']
    df['packet_size'] = df['TotLen Fwd Pkts'] + df['TotLen Bwd Pkts']

    # Avoid Division by 0
    df['flow_duration'] = df['flow_duration'].replace(0, 1e-6)

    # Calculate Derived Features
    df['packet_rate'] = df['packet_count'] / df['flow_duration']
    df['byte_rate'] = df['packet_size'] / df['flow_duration']

    # Return only relevant columns
    return df.loc[:, ['packet_size', 'packet_rate', 'byte_rate']]

In [12]:
benign_features = extract_features(df_benign)
benign_features.head()

,packet_size,packet_rate,byte_rate
0,0,0.026633,0.000000
1,0,0.026633,0.000000
2,0,0.026634,0.000000
3,3512,3.873587,544.161528
4,3352,2.839597,380.733175


**Create Full Dataset**:

In [13]:
def preprocess_file(filepath):
    # Load the CSV
    df = pd.read_csv(filepath)

    # Filter only normal activities
    df_benign = df[df['Label'].str.strip().str.lower() == 'benign'].copy()

    # Extract the features
    df_features = extract_features(df_benign)

    return df_features

In [14]:
all_features = []

for filename in os.listdir('../dataset/'):
    if filename.endswith('.csv'):
        path = os.path.join('../dataset', filename)
        print(f'[+] Processing: {filename}')
        features = preprocess_file(path)
        all_features.append(features)

# Combine All
final_df = pd.concat(all_features, ignore_index=True)

[+] Processing: 02-14-2018.csv
[+] Processing: 02-15-2018.csv
[+] Processing: 02-16-2018.csv


C:\Users\emrev\AppData\Local\Temp\ipykernel_20436\386629215.py:3: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


[+] Processing: 02-20-2018.csv
[+] Processing: 02-21-2018.csv
[+] Processing: 02-22-2018.csv
[+] Processing: 02-23-2018.csv
[+] Processing: 02-28-2018.csv


C:\Users\emrev\AppData\Local\Temp\ipykernel_20436\386629215.py:3: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


[+] Processing: 03-01-2018.csv


C:\Users\emrev\AppData\Local\Temp\ipykernel_20436\386629215.py:3: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


[+] Processing: 03-02-2018.csv


In [16]:
len(final_df)

13484708

## Model Training

In [18]:
model = IsolationForest(contamination=0.1, random_state=42)
model.fit(final_df)

IsolationForest(contamination=0.1, random_state=42)

In [19]:
df = pd.read_csv('../dataset/02-14-2018.csv')
df_attack = df[df['Label'].str.strip().str.lower() != 'benign'].copy()

In [20]:
df_attack.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
94,21,6,14/02/2018 10:33:26,19,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,FTP-BruteForce
95,21,6,14/02/2018 10:33:26,3,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,FTP-BruteForce
96,21,6,14/02/2018 10:33:26,3,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,FTP-BruteForce
97,21,6,14/02/2018 10:33:26,2,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,FTP-BruteForce
98,21,6,14/02/2018 10:33:26,2,1,1,0,0,0,0,...,40,0.0,0.0,0,0,0.0,0.0,0,0,FTP-BruteForce


In [21]:
df_attack['Label'].nunique()

2

In [22]:
attack_features = extract_features(df_attack)

In [23]:
preds = model.predict(attack_features)

In [24]:
anomalies = np.sum(preds == -1)
print(f'Detected {anomalies} anomalies out of {len(preds)} attack samples.')

Detected 272983 anomalies out of 380949 attack samples.


## Model Saving

In [26]:
joblib.dump(model, '../models/isolation_forest.joblib')

['../models/isolation_forest.joblib']